In [1]:
!sudo apt-get update
!pip install crewai
!pip install 'crewai[tools]'
!pip install google-api-python-client
!pip install langchain-openai


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,318 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://security.ubuntu.com/ub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.3 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=30a5cdf4668d1efe007b38a455d47adf59320df59c4cf7bcf7c98cf882bbfde1
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
  Attempting uninstall: pytest
    Foun

In [1]:
# Import our API Keys
import os
from google.colab import userdata

# Keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')  # Google Custom Search Engine ID
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [7]:
# Set the Topic Here
#######
search_query = """
Demystifing Autonomous AI Agents in the Digital Age.  A deeper look at Agentic, Generative, Large Language Models (LLM) and Large Action Models (LAM).
"""
####

In [8]:
import os
import json
from googleapiclient.discovery import build
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool


# Define Google Search Tool
class GoogleSearchTool:
    def __init__(self, api_key, cse_id):
        if not api_key or not cse_id:
            raise ValueError("API key and CSE ID must be provided")

        # Initialize the Google Custom Search API client
        self.api_key = api_key
        self.cse_id = cse_id
        self.service = build("customsearch", "v1", developerKey=self.api_key)
        self.name = "GoogleSearchTool"
        self.args = {"query": "Search term to find relevant results"}
        self.description = "Performs a Google search and returns relevant results."

    def search(self, query, num_results=5):
        if not query or not isinstance(query, str):
            return "Error: Invalid query provided."

        try:
            # Execute the search query
            results = self.service.cse().list(q=query, cx=self.cse_id, num=num_results).execute()
            items = results.get('items', [])
            return [
                {"title": item.get('title', 'No title'), "url": item.get('link', 'No link'), "snippet": item.get('snippet', 'No snippet')}
                for item in items
            ]
        except Exception as e:
            return f"Error during search: {str(e)}"

        # Add invoke method to handle CrewAI's expectations
    def invoke(self, input=None):
        try:
            # Parse the input, it is expected to be a JSON string or dict
            input_data = json.loads(input) if isinstance(input, str) else input
            query = input_data.get("query")

            if not query:
                return "Error: No query provided."

            # Perform the search
            results = self.search(query)
            return json.dumps(results, indent=2)
        except json.JSONDecodeError:
            return "Error: Failed to parse input as JSON."
        except Exception as e:
            return f"Error: {str(e)}"


# Define the Research Agent
class ResearchAgent(Agent):
    def __init__(self, name, role, goal, backstory, tools):
        super().__init__(name=name, role=role, goal=goal, backstory=backstory, tools=tools)

    # Method to process search results and extract key insights
    def process_search_results(self, search_results):
        insights = []
        for result in search_results:
            insights.append(f"Title: {result['title']}\nSummary: {result['snippet']}\nURL: {result['link']}")
        return "\n".join(insights)


# Initialize Google Search Tool with API keys
google_search_tool = GoogleSearchTool(
    api_key=os.getenv("GOOGLE_API_KEY"),
    cse_id=os.getenv("GOOGLE_CSE_ID")
)

# Define Agents
researcher = ResearchAgent(
    name='Principal Researcher and Analyst',
    role='Researcher',
    goal=f"Uncover cutting-edge developments in {search_query} and related topics.",
    backstory="""
        You are an experienced and award-winning researcher who excels at finding actionable insights
        and translating complex data into engaging content.
    """,
    tools=[google_search_tool, ScrapeWebsiteTool()]
)

writer = Agent(
    role='Tech Content Strategist',
    goal=f"Craft compelling content on {search_query} advancements.",
    backstory="""
      You are a renowned Content Strategist known for translating complex ideas into engaging narratives.
    """,
    verbose=True,
    allow_delegation=True,
    llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
)

# Define Tasks
def researcher_task_execution():
    # Researcher performs the search and processes the results
    # search_query = "The latest trends in AI and Data Analytics 2024"
    search_results = google_search_tool.search(query=search_query)

    # Researcher processes search results to extract insights
    processed_insights = researcher.process_search_results(search_results)

    # Save processed insights to a file (or return for next task)
    with open("researcher_tasks.md", "w") as file:
        file.write(processed_insights)

    return processed_insights


# Research Task: Perform the search, process the results, and save insights
research = Task(
    description=f"""
      Extract key insights, ideas, and information from {search_query}.
    """,
    expected_output=f"""
      A concise report on {search_query}, containing key insights and recommendations in bullet points.
    """,
    agent=researcher,
    output_file="researcher_tasks.md",
    execution_function=researcher_task_execution  # Executes the researcher's function to gather insights
)


# Writing Task: Use the processed insights to write a blog post
def writer_task_execution(context):
    # Retrieve the insights from the previous research task (context)
    insights = context  # passed from research task

    # Writer generates the blog post using insights
    blog_post = f"""
    # {search_query}

    In this blog, we explore the latest trends:

    {insights}

    Stay tuned for more cutting-edge developments.
    """

    # Return the blog post (CrewAI will handle saving to output_file)
    return blog_post


write_blog = Task(
    description=f"""
    Thought: Review the research findings on {search_query} and think about how to structure an engaging blog post.
    Action: Use the research data to write a compelling 800-word blog post, including relevant citations from the URLs provided in the research.
    Observation: Make sure the post is coherent, flows well, and stays on topic. Ensure all URLs are properly cited.
    """,
    expected_output="""
      A full blog post of around 800 words with citations from all the URLs.
    """,
    agent=writer,
    context=[research],  # Use the output of the research task as context
    output_file="writer_tasks.md",  # CrewAI will save the blog post to this file
    execution_function=writer_task_execution
)



# Instantiate the Crew and kick off the process
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write_blog],
    verbose=True,
    process=Process.sequential  # Ensure the research task finishes before writing starts
)

# Kickoff the crew process
result = crew.kickoff()

# Print the result (final blog post)
print(result)


# Agent: Researcher
## Task: 
      Extract key insights, ideas, and information from 
Demystifing Autonomous AI Agents in the Digital Age.  A deeper look at Agentic, Generative, Large Language Models (LLM) and Large Action Models (LAM).
.
    


# Agent: Researcher
## Thought: I need to gather information on "Demystifying Autonomous AI Agents in the Digital Age," focusing on Agentic, Generative, Large Language Models (LLM), and Large Action Models (LAM). I will start by performing a Google search to find relevant content.
## Using tool: GoogleSearchTool
## Tool Input: 
"{\"query\": \"Demystifying Autonomous AI Agents in the Digital Age Agentic Generative Large Language Models LLM Large Action Models LAM\"}"
## Tool Output: 
[
  {
    "title": "tmgthb/Autonomous-Agents: Autonomous Agents (LLMs ... - GitHub",
    "url": "https://github.com/tmgthb/Autonomous-Agents",
    "snippet": "2nd of July 2024. Coalitions of Large Language Models Increase the Robustness of AI Agents. Coalition of L